In [1]:
import sys
sys.path.append('..')

from utils.llm_generator import LLMGenerator
# API配置
api_config = {
    "model_type": "api",
    "model_name": "qwen2.5_7b",
    "api_key": "sk-zk25bb059bd482cb639fde4e4d8d1b524dd663f67cb7bbbd",  # 替换为你的API密钥
    "base_url": "http://localhost:11434/v1"  # 替换为实际的API地址
}

# 创建LLM生成器实例
llm = LLMGenerator(**api_config)


messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "你是谁，你的"},
]


response = llm.generate(messages)
print(response)

2025-03-20 20:49:55 - root - INFO - === Amplifier Design System Started ===
2025-03-20 20:49:55 - root - INFO - Log file created at: logs/amp_design_20250320_204955.log
2025-03-20 20:49:55 - LLMGenerator - INFO - 开始生成响应...
2025-03-20 20:50:28 - httpx - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-20 20:50:28 - LLMGenerator - INFO - 成功生成响应
2025-03-20 20:50:28 - LLMGenerator - INFO - 生成的响应: 我是阿里云开发的一款超大规模语言模型，我叫通义千问。很高兴能为你解答问题和提供帮助！


我是阿里云开发的一款超大规模语言模型，我叫通义千问。很高兴能为你解答问题和提供帮助！


In [1]:
import re

# 定义网表内容
netlist = """
.subckt AMP Vinp Vinn VDD VSS Vout
XM1 net1 Vinn net3 VSS sky130_fd_pr__nfet_01v8 L=1 W=30 m=1
XM2 net2 Vinp net3 VSS sky130_fd_pr__nfet_01v8 L=1 W=30 m=1
I0 net3 VSS 50p
XM3 net1 net1 VDD VDD sky130_fd_pr__pfet_01v8 L=1 W=64 m=1
XM4 net2 net1 VDD VDD sky130_fd_pr__pfet_01v8 L=1 W=64 m=1
XM5 Vout net2 VSS VSS sky130_fd_pr__nfet_01v8 L=1 W=300 m=1
I1 VDD Vout 50p
Cc net2 Vout 1p m=1
.ends AMP
"""

# 正则表达式匹配 MOSFET 的 L 和 W 参数
pattern = r"XM\d+\s+\S+\s+\S+\s+\S+\s+\S+\s+sky130_fd_pr__\w+\s+L=([\d.]+)\s+W=([\d.]+)"

# 查找所有匹配的 L 和 W 值
matches = re.findall(pattern, netlist)

# 定义参数范围
min_value = 0.2
max_value = 100

# 检查每个 L 和 W 是否在范围内
for idx, (L, W) in enumerate(matches, start=1):
    L = float(L)
    W = float(W)
    print(f"Transistor XM{idx}: L={L}, W={W}")
    
    if min_value <= L <= max_value and min_value <= W <= max_value:
        print("  L and W are within the valid range [0.2, 100].")
    else:
        print("  ERROR: L or W is out of the valid range [0.2, 100]!")

Transistor XM1: L=1.0, W=30.0
  L and W are within the valid range [0.2, 100].
Transistor XM2: L=1.0, W=30.0
  L and W are within the valid range [0.2, 100].
Transistor XM3: L=1.0, W=64.0
  L and W are within the valid range [0.2, 100].
Transistor XM4: L=1.0, W=64.0
  L and W are within the valid range [0.2, 100].
Transistor XM5: L=1.0, W=300.0
  ERROR: L or W is out of the valid range [0.2, 100]!


In [8]:
import numpy as np

# 示例数据：每个网表的仿真结果
netlists = [
    {"name": "Netlist1", "Gain": 60, "GBW": 100, "SR": 200, "PM": 45, "Idc": 10},
    {"name": "Netlist2", "Gain": 70, "GBW": 120, "SR": 220, "PM": 50, "Idc": 12},
    {"name": "Netlist3", "Gain": 80, "GBW": 150, "SR": 250, "PM": 55, "Idc": 15},
]

# 计算每个指标的最大值和最小值
Gainmax = max(net["Gain"] for net in netlists)
GBWmax = max(net["GBW"] for net in netlists)
SRmax = max(net["SR"] for net in netlists)
PMmax = max(net["PM"] for net in netlists)
Idcmax = max(net["Idc"] for net in netlists)

Gainmin = min(net["Gain"] for net in netlists)
GBWmin = min(net["GBW"] for net in netlists)
SRmin = min(net["SR"] for net in netlists)
PMmin = min(net["PM"] for net in netlists)
Idcmin = min(net["Idc"] for net in netlists)

# 计算每个网表的得分
for net in netlists:
    # 基础分计算
    Gain_base = 12 if net["Gain"] >= 60 else (net["Gain"] / Gainmax) * 12
    GBW_base = 12 if net["GBW"] >= 20 else (net["GBW"] / GBWmax) * 12
    SR_base = 12 if net["SR"] >= 20 else (net["SR"] / SRmax) * 12
    PM_base = 12 if net["PM"] >= 60 else (net["PM"] / PMmax) * 12
    Idc_base = 12 if net["Idc"] <= 500 else (Idcmax / net["Idc"]) * 12

    # 额外分计算
    P_Idc = (1 - (net["Idc"] - Idcmin) / (Idcmax - Idcmin)) * 6
    P_Gain = ((net["Gain"] - Gainmin) / (Gainmax - Gainmin)) * 4.5
    P_GBW = ((net["GBW"] - GBWmin) / (GBWmax - GBWmin)) * 4.5
    P_SR = ((net["SR"] - SRmin) / (SRmax - SRmin)) * 4.5
    P_PM = ((net["PM"] - PMmin) / (PMmax - PMmin)) * 3

    # 总得分
    total_score = (
        Gain_base + GBW_base + SR_base + PM_base + Idc_base +
        P_Idc + P_Gain + P_GBW + P_SR + P_PM
    )

    net["score"] = total_score

# 按得分排序并挑选最好的网表
best_netlist = max(netlists, key=lambda x: x["score"])

# 输出结果
print("Best Netlist:")
print(best_netlist)

Best Netlist:
{'name': 'Netlist3', 'Gain': 80, 'GBW': 150, 'SR': 250, 'PM': 55, 'Idc': 15, 'score': 76.5}
